In [ ]:
import os
import sys

cwd=os.path.dirname(os.getcwd())
sys.path.append(cwd)
print(cwd)

from oligo_designer_toolsuite.oligo_specificity_filter import Blastn, Bowtie, Bowtie2, ExactMatches, BowtieSeedRegion, LigationRegionCreation, SeedRegionCreationStandard, SeedRegionCreationPercentage

from oligo_designer_toolsuite.utils._data_parser import read_oligos_DB_tsv
from oligo_designer_toolsuite.IO._database import CustomDB
from oligo_designer_toolsuite.oligo_specificity_filter._specificity_filters import SpecificityFilter

In [ ]:
n_jobs=10
ligation_region=10
dir_output=cwd + "/output"
dir_annotations=dir_output +"/reference"
dir_specificity = dir_output + "/specificity_temporary"
dir_probes = dir_output +"/property_filter"
file_probe_info=dir_probes + "/oligo_DB_human_GRCh38.p14_NCBI_release_GCF_000001405.40_gene_transcript.tsv"
print(file_probe_info)
min_probes_per_gene=2
file_transcriptome_fasta=dir_annotations + "/reference_DB_human_GRCh38.p14_NCBI_release_GCF_000001405.40_gene_transcript.fna"
word_size=10
percent_identity=80
probe_length_min=30
probe_length_max=40
coverage=50
seed_region_creation = LigationRegionCreation(ligation_region_size=ligation_region)


In [ ]:
probe_info_dict=read_oligos_DB_tsv(file_probe_info)
for gene in probe_info_dict.keys():
    print(f"{gene}, {len(list(probe_info_dict[gene].keys()))}")

In [ ]:
bowtie2_filter=Bowtie2(dir_specificity)

probe_info_dict=read_oligos_DB_tsv(file_probe_info)
probes_updated = bowtie2_filter.apply(probe_info_dict, file_transcriptome_fasta, n_jobs)
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")

Test exact match filter

In [ ]:
exact_match_filter=ExactMatches(dir_specificity)
#exact_match_filter=ExactMatches(n_jobs, dir_output, dir_annotations)

In [ ]:
probes_updated = exact_match_filter.apply(probe_info_dict, file_transcriptome_fasta, 10)
#probes_updated = exact_match_filter.apply(probe_info_dict)
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")

Test Blast filter

In [ ]:
blast_filter=Blastn(dir_specificity, word_size,percent_identity, coverage)

In [ ]:
probes_updated = blast_filter.apply(probe_info_dict, file_transcriptome_fasta, n_jobs)
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")
print(probes_updated["NOC2L"])

Test the two filters one after the other

In [ ]:
dir_annotation = '/home/francesco/Desktop/Work/NCBI'
annotation = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.gtf'
sequence = dir_annotation+'/GCF_000001405.40_GRCh38.p14_genomic.fna'
ligation_seed_region = LigationRegionCreation(ligation_region_size=10)
seed_region_filter = BowtieSeedRegion(dir_specificity, ligation_seed_region)
db = CustomDB(25, 30, None, dir_output=dir_output, file_annotation=annotation, file_sequence = sequence)
db.read_reference_DB(file_reference_DB=file_transcriptome_fasta)
db.read_oligos_DB(file_oligos_DB_tsv=file_probe_info, format='tsv')
specificity_filters = SpecificityFilter([exact_match_filter,seed_region_filter, blast_filter])
db = specificity_filters.apply(database=db, n_jobs=n_jobs)
for gene in db.oligos_DB.keys():
    print(f"{gene}, {len(list(db.oligos_DB[gene].keys()))}")

Test Bowtie filter

In [ ]:
min_mismatches=3
mismatch_region=5
ligation_region=0

bowtie_filter=Bowtie(dir_specificity, min_mismatches, mismatch_region)

probe_info_dict=read_oligos_DB_tsv(file_probe_info)
probes_updated = bowtie_filter.apply(probe_info_dict, file_transcriptome_fasta, n_jobs)
#find a probe with mismatch in ligation region and run on that. 
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")



Test Bowtie 2 filter

In [ ]:
bowtie2_filter=Bowtie2(dir_specificity)

probe_info_dict=read_oligos_DB_tsv(file_probe_info)
probes_updated = bowtie2_filter.apply(probe_info_dict, file_transcriptome_fasta, n_jobs)
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")

In [ ]:
ligation_seed_region = LigationRegionCreation(ligation_region_size=10)
standard_seed_region = SeedRegionCreationStandard(start=5, end = 25)
prec_seed_region = SeedRegionCreationPercentage(start=0.2, end = 0.8)

bowtie_seed_region = BowtieSeedRegion(dir_specificity, ligation_seed_region)

probe_info_dict=read_oligos_DB_tsv(file_probe_info)
probes_updated = bowtie_seed_region.apply(probe_info_dict, file_transcriptome_fasta, n_jobs)
for gene in probes_updated.keys():
    print(f"{gene}, {len(list(probes_updated[gene].keys()))}")